In [8]:
import sys
import os
from datetime import datetime
import json
import shutil
import random

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tqdm import tqdm_notebook, tqdm
from matplotlib import pyplot as plt

sys.path.append('../src/')
from env import BTCMarket_Env
from agent import Trader_Agent
from DQNtrainer import DQNTrainer
from collections import deque
import h5py


In [2]:
obs_space = (5,20)
act_space = 4

money = 200
fee = 0.001
episodes = 1
runs_p_eps = 1

env = BTCMarket_Env(observation_space = obs_space,
            action_space = act_space,
            start_money = money,
            trading_fee= fee)
agent = Trader_Agent(observation_space = obs_space,
            action_space = act_space,
            epsilon = 0.01)
dqntrainer = DQNTrainer(env, agent,
            observation_space = obs_space,
            action_space = act_space,
            batch_size=32)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 256)               25856     
                                                                 
 dense_1 (Dense)             (None, 128)               32896     
                                                                 
 dense_2 (Dense)             (None, 64)                8256      
                                                                 
 dense_3 (Dense)             (None, 4)                 260       
                                                                 
Total params: 67,268
Trainable params: 67,268
Non-trainable params: 0
_________________________________________________________________


2023-02-15 10:31:07.461328: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-15 10:31:07.462181: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [3]:
env.reset()
data_samples = env.episode_length
memory = deque()

state, _, _ = env.step(np.array([0]))
for t in tqdm(range(data_samples)):
    # Compute Action
    tmp_wallet_value = env.wallet_value
    action = agent.compute_action(state)
    # Transform Action from Policy to Env Requirement 
    dqn_action = dqntrainer.transforme_to_dqn_action(action)
    # Compute new step
    next_state, reward, done = env.step(action=dqn_action)
    # save Experience to Memory
    memory.append((state, action, reward, next_state, done))
    state = next_state

100%|█████████████████████████████████████████| 671/671 [01:12<00:00,  9.27it/s]


In [31]:
batch_size = 50


batch = []
# for i in range(1):
for i in range(max([len(memory) - batch_size,0]), len(memory)):
    batch.append(memory[i])

# Change here to fit new RL-Algo
# for index in range(0,len(batch)-1):

x_train = np.zeros((batch_size, 100))
y_train = np.zeros((batch_size, 4))
print(len(batch))


state, _, reward, next_state , done = batch[0]
action = agent.model.predict(state,verbose = 0)
for index in range(1,len(batch)):
    # Compute Reward Decay for DQN
    action_next = agent.model.predict(next_state,verbose = 0)
    if not done:
        reward += dqntrainer.gamma * np.max(action_next)
        
    if np.any(np.isnan(reward)) or np.any(np.isnan(action)):
        print(reward)
        print(action)
    target = np.nan_to_num(action,nan=0.0)      
    id_act = np.argmax(target)
    target[0,id_act] = reward
    
    target = np.nan_to_num(target,nan=0.0)
    state = np.nan_to_num(state,nan=0.0)
    x_train[index]= state[0]
    y_train[index]= target[0]
    
    
    # update state, action, reward for training
    state, _, reward, next_state , done = batch[index]
    action = action_next


result=dqntrainer.agent.model.fit(x_train, y_train, epochs=5, verbose=1)
# agent.update_epsilon()


50
Epoch 1/5
2/2 [==============================] - 0s 15ms/step - loss: 0.2304
Epoch 2/5
2/2 [==============================] - 0s 13ms/step - loss: 0.2303
Epoch 3/5
2/2 [==============================] - 0s 16ms/step - loss: 0.2303
Epoch 4/5
2/2 [==============================] - 0s 15ms/step - loss: 0.2303
Epoch 5/5
2/2 [==============================] - 0s 14ms/step - loss: 0.2303


In [32]:
y_train[-1]

array([0.45314425, 0.4207291 , 1.96313   , 0.50530088])

In [30]:
index

48

In [ ]:
tmp[0].shape

In [ ]:
tmp = tmp.reshape((100))
tmp.shape